# ![](https://ga-dash.s3.amazonaws.com/production/assets/logo-9f88ae6c9c3871690e33280fcf557f33.png) Capstone Project: Donor Leads through Networks


--- 

By: Wenzhe

## Network Clustering and Graphing

### Overview

The goal of this notebook is cluster nodes of the network by using Community Detection algorithms, and to visualise the graphs based on its clusters.

### Notebook Structure

* [Part 1: Setup](#part-1-eda)
* [Part 2: Community Detection](#part-2-community-detection)
* [Part 3: Evaluation Metrics](#part-3-evaluation-metrics)
* [Part 4: Summary](#part-4-summary)

---

## Part 1: Setup

#### Import Libraries

In [1]:
# conda install conda-forge::karateclub

In [2]:
#!pip install karateclub

In [3]:
#!pip install cdlib

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# NetworkX to create and work with Networks
import networkx as nx

# Pyvis to visualise the graps from NetworkX 
from pyvis.network import Network

# To write the graph data into JSON
import json
from networkx.readwrite import json_graph

# For forest centrality
import networkit

# For Community Detection
import karateclub
from cdlib import algorithms
from cdlib import evaluation

# For making copies
import copy

Note: to be able to use all crisp methods, you need to install some additional packages:  {'graph_tool', 'bayanpy', 'leidenalg', 'wurlitzer', 'infomap'}
Note: to be able to use all crisp methods, you need to install some additional packages:  {'ASLPAw', 'pyclustering'}
Note: to be able to use all crisp methods, you need to install some additional packages:  {'wurlitzer', 'infomap', 'leidenalg'}


### Import JSON Graph

Load the previously saved graph that is store in JSON format.

In [5]:
with open('../json/graph.json', "r") as f:
    json_data = json.load(f)

In [6]:
G = json_graph.node_link_graph(json_data)

In [7]:
# Check that graph has been loaded
len(G.nodes())

24756

---

## Part 2: Community Detection

Clustering in machine learning refers to the grouping of data points based on their attributes. Community detection is specific for network analysis where it is based on the single attribute of edges. Community detection methods broadly fall into **Agglomerative Methods** and **Divisive Methods**.

Source: https://towardsdatascience.com/community-detection-algorithms-9bd8951e7dae

The network derived from the dataset is disjoint, undirected and unweighted. In this section, explore various community detection models that are suitable for this network.

This will be primarily unsupervised, with some of the community subgraphs visualised to inspect the communities formed. The goal of this is to find a suitable model that is likely to group organisations and people based on the network.

Before that, some helper functions:

### Visualisation Helper Functions

In [8]:
# Returns the name of person / charity
def label_node_name(node):
    if node.get('node-type') == 'person':
        return node.get('name')
    else:
        return node.get('charity_name')

In [9]:
# make persons dot and charities box on the visualisation
def label_node_shape(node):
    if node['node-type'] == 'person':
        return 'dot'
    else:
        return 'box' # square

In [10]:
def label_node_color_degree(graph, node, start_node):
    path_len = nx.shortest_path_length(graph, source=start_node, target=node)
    if path_len == 0: path_len = 1 # if target is same as source, set it to be the warmest color (same as deg=1)
    if path_len > 6: path_len = 6 # this color palette only has 6 colours, limit anything beyond 6 to use the coolest color available
    return sns.color_palette('coolwarm').as_hex()[-path_len]

In [11]:
sns.color_palette('coolwarm').as_hex()[-1]

'#e26952'

In [12]:
def visualise_graph(graph, filename):
    for n in graph.nodes(data=True):
        n[1]['label'] = label_node_name(n[1])
        n[1]['shape'] = label_node_shape(n[1])
    
    pyvis_net = Network(notebook=True, cdn_resources='remote') 
    pyvis_net.from_nx(graph) 
    pyvis_net.show(f'../graph/{filename}.html') 

In [13]:
# Returns the node of a charity based on its name
# Names are unique, else returns the first instance
def get_charity_by_name(G, name):
    node_select = [node for node, data in G.nodes(data=True) if data.get('node-type') == 'entity' and data.get('charity_name') == name]
    # return none if no such charity name
    if len(node_select) == 0: return None
    return node_select[0]

In [14]:
# Returns the node of a person based on the name
def get_person_by_name(G, name):
    node_select = [node for node, data in G.nodes(data=True) if data.get('node-type') == 'person' and data.get('name') == name]
    # return none if no such charity name
    if len(node_select) == 0: return None
    return node_select[0]

In [15]:
def visualise_degree_graph(graph, charity_name, degree, filename):
    charity_node = get_charity_by_name(graph, charity_name)
    subgraph = nx.ego_graph(graph, charity_node, degree)
    for n in subgraph.nodes(data=True):
        n[1]['label'] = label_node_name(n[1])
        n[1]['shape'] = label_node_shape(n[1])
        n[1]['color'] = label_node_color_degree(subgraph, n[0], charity_node)
        # TODO: label entity features
    
    pyvis_net = Network(notebook=True, cdn_resources='remote') 
    pyvis_net.from_nx(subgraph) 
    pyvis_net.show(f'../graph/{filename}.html') 

In [16]:
visualise_degree_graph(G, 'IMPART LTD.', 5, 'impart_deg5')

../graph/impart_deg5.html


### Creating a Charity only Network

The current network graph only contains edges between people and entity nodes. This means there are no people-people or entity-entity edges. We explore whether there will be a difference in community detection if simplification of the network is done and only entity nodes remain, and replacing people nodes witih their entity connections.

In [17]:
# Create a new network graph only for charity nodes
G_charity_only = nx.Graph()

In [18]:
for node, data in G.nodes(data=True):
    # Add charity node to the new graph
    if data.get('node-type') == 'entity':
        G_charity_only.add_node(node, **data)
    # Get the edges for person node and connect the charities instead    
    else:
        # Add edges between entities if person has more than one edge
        # This implies that the person is connected to more than one charity
        num_edges = len(G.edges(node))
        if num_edges > 1:
            node_edge_list = list(G.edges(node))
            from_charity_name = data.get('charity_name')
            from_charity_node = get_charity_by_name(G, from_charity_name)
            for i in range(num_edges):
                other_node = node_edge_list[i][1]
                if other_node != from_charity_node:
                    G_charity_only.add_edge(from_charity_node, other_node)
                    print(f"From person {data.get('name')}: Connecting {G.nodes()[from_charity_node]['charity_name']} to {G.nodes()[other_node]['charity_name']}")
                # for j in range(i+1, num_edges):
                #     G_charity_only.add_edge(node_edge_list[i][1], node_edge_list[j][1])
                #     print(f"From person {data.get('name')} Connecting {G.nodes()[node_edge_list[i][1]]['charity_name']} to {G.nodes()[node_edge_list[j][1]]['charity_name']}")
            

From person ZHOU LIHAN: Connecting #CHECKED LIMITED to OnePeople.sg
From person TAN LIN TECK: Connecting *SCAPE CO., LTD. to All Saints Home
From person TAN LIN TECK: Connecting *SCAPE CO., LTD. to Nanyang Polytechnic
From person TAN LIN TECK: Connecting *SCAPE CO., LTD. to Singapore Youth Flying Club
From person TAN LIN TECK: Connecting *SCAPE CO., LTD. to TELOK AYER CHINESE METHODIST CHURCH AND TELOK AYER CHINESE METHODIST CHURCH (TA2 SANCTUARY)
From person CHUA DAVID: Connecting *SCAPE CO., LTD. to CYBER YOUTH SG LTD.
From person CHUA DAVID: Connecting *SCAPE CO., LTD. to MOUNT CARMEL BP CHURCH LIMITED
From person CHUA DAVID: Connecting *SCAPE CO., LTD. to National Youth Fund
From person CHUA DAVID: Connecting *SCAPE CO., LTD. to SINGAPORE INSTITUTE OF MANAGEMENT GROUP LIMITED
From person CHUA DAVID: Connecting *SCAPE CO., LTD. to SINGAPORE UNIVERSITY OF SOCIAL SCIENCES
From person WONG SOK YEE: Connecting *SCAPE CO., LTD. to VARIETY - THE CHILDREN'S CHARITY OF SINGAPORE LIMITED
Fro

In [19]:
len(G_charity_only.nodes())

2601

In [20]:
# remap the labels of nodes to be continuous to allow for modelling
mapping = {node: i for i, node in enumerate(G_charity_only.nodes())}
G_charity_only = nx.relabel_nodes(G_charity_only, mapping)

In [21]:
# Save the graph
graph_data = json_graph.node_link_data(G_charity_only)
with open('../json/graph_charity.json', "w") as f:
    json.dump(graph_data, f, indent=4)

In [22]:
visualise_degree_graph(G_charity_only, 'IMPART LTD.', 3, 'impart_deg3_char')

../graph/impart_deg3_char.html


---

### Ego-Splitting Framework

This algorithm is able to split the network by making persona graphs, and accounts for overlaps where a single node may have multiple personas. This method is suitable for overlapping and non-overlapping cluster memberships. It involves two steps:

1) **Local Step**: Partition the nodes' ego-nets with a partitioning algorithm, then split each node into its persona nodes in its communities. This is done using the Louvian method, which is a method for extracting non-overlapping communities from large networks.
2) **Global Step**: Partition the new graph to get overlapping clusters of the original graph.

Source: [KDD 2017 Research Paper](https://www.eecs.yorku.ca/course_archive/2017-18/F/6412/reading/kdd17p145.pdf)

We run this algorithm on both the original network as well as the charity only network to visually inspect for differences.

In [23]:
ENS = karateclub.EgoNetSplitter(weight=None)
ENS_c = karateclub.EgoNetSplitter(weight=None)

As this method alters the original graph, use a deep copy of the graph to run this method and use the communities created to visualise on the original graph.

In [24]:
G_ens = copy.deepcopy(G)
G_ens_c = copy.deepcopy(G_charity_only)
ENS.fit(G_ens)
ENS_c.fit(G_ens_c)

In [25]:
communities = ENS.get_memberships()
communities_c = ENS_c.get_memberships()

As this algorithm caters for overlapping clusters, check whether any node has been clustered under multiple communities.

In [26]:
for node, community in communities.items():
    if len(community) > 1:
        print(f"Node {node}: Community {community}")

In [27]:
for node, community_c in communities_c.items():
    if len(community_c) > 1:
        print(f"Node {node}: Community {community_c}")

For this network, each node only belongs to one community.

In [28]:
def visualise_subgraph(subgraph, charity_node, filename):
    for n in subgraph.nodes(data=True):
        n[1]['label'] = label_node_name(n[1])
        n[1]['shape'] = label_node_shape(n[1])
        n[1]['color'] = label_node_color_degree(subgraph, n[0], charity_node)
        # TODO: label entity features
    
    pyvis_net = Network(notebook=True, cdn_resources='remote') 
    pyvis_net.from_nx(subgraph) 
    pyvis_net.show_buttons(filter_=['physics'])
    pyvis_net.show(f'../graph/{filename}.html') 

In [29]:
def get_community_subgraph_visualise(graph, charity_name, communities, filename):
    charity_node = get_charity_by_name(graph, charity_name)
    community_num = list(communities.items())[charity_node][1][0]
    community_nodes = [node for node, community in communities.items() if community[0] == community_num]
    subgraph = graph.subgraph(community_nodes)

    visualise_subgraph(subgraph, charity_node, filename)
    return 
    

In [30]:
get_community_subgraph_visualise(G, 'IMPART LTD.', communities, 'impart_community')
get_community_subgraph_visualise(G_charity_only, 'IMPART LTD.', communities_c, 'impart_char_community')

../graph/impart_community.html
../graph/impart_char_community.html


In [31]:
get_community_subgraph_visualise(G, 'ACMI Migrant Fund', communities, 'acmi_community')
get_community_subgraph_visualise(G_charity_only, 'ACMI Migrant Fund', communities_c, 'acmi_char_community')

../graph/acmi_community.html
../graph/acmi_char_community.html


Visually inspecting the graphs, the communities generated on both networks differ significantly. Communities on the charity only network contain much more charity nodes, while the communities for the original network is much smaller.

The boundaries of communities can be completely different as seen in the example above.

In [32]:
person_node = get_person_by_name(G, 'WILLIAM GOH')
G.nodes()[person_node]

{'name': 'WILLIAM GOH',
 'role': 'Patron',
 'designation': nan,
 'charity_name': 'ACMI Migrant Fund',
 'charity_uen': 'T16CC0006L',
 'node-type': 'person',
 'label': 'WILLIAM GOH',
 'shape': 'dot'}

In [33]:
person_community_num = list(communities.items())[person_node][1][0]
person_community_num

42

In [34]:
person_subgraph_nodes = [node for node, community in communities.items() if community[0] == person_community_num]

In [35]:
visualise_subgraph(G.subgraph(person_subgraph_nodes), person_node, 'person_community')

../graph/person_community.html


In [36]:
visualise_degree_graph(G, 'ACMI Migrant Fund', 4, 'acmi_deg4')

../graph/acmi_deg4.html


The person examined in the graph is connected to ACMI Migrant Fund, is also connected to many Catholic churches in the position of Archbishop. This shows that the EgoSplitter algorithm is able to cluster the Archbishop with the Catholic churches; whereas ACMI Migrant Fund itself is clustered with several other Catholic organisations. Intuitively this clustering has been able to separate the churches from other charity organisations in the Catholic community.

In [37]:
get_community_subgraph_visualise(G, 'St James\' Church', communities, 'stjames_community')
get_community_subgraph_visualise(G_charity_only, 'St James\' Church', communities_c, 'stjames_char_community')   

../graph/stjames_community.html
../graph/stjames_char_community.html


### Divide and Conquer Strategy

This algorithm works by splitting the graphs into multiple modules and then detect communities within each module. It then selects a set of local leaders from each module and then expanded to form communites around the leader nodes.

Source: [DCS: Divide and Conquer Strategy for Detecting Overlapping Communities in Social Graphs](https://github.com/SyedAgha/Divide-and-Conquer/blob/master/DCS_code_and_paper/DCS.pdf)

In [38]:
G_copy = copy.deepcopy(G)

In [39]:
coms = algorithms.dcs(G_copy)

In [40]:
coms.communities

[[1123,
  3737,
  7433,
  8760,
  8761,
  8762,
  8763,
  8764,
  8765,
  8766,
  8767,
  8768,
  8769,
  8770,
  8771,
  8772,
  8773,
  8774,
  8775,
  8776,
  8777,
  8778,
  8779,
  8780,
  8781,
  8782,
  8783,
  8784,
  8786,
  8787,
  8788,
  8789,
  8790,
  8791,
  8792,
  8793,
  8794,
  8795,
  8796,
  8797,
  8798,
  8799,
  8800,
  8801,
  8802,
  8803,
  8804,
  8805,
  8806,
  8808,
  8809,
  8810,
  8811,
  8812,
  8813,
  8814,
  8815,
  8816,
  8817,
  8818,
  8819,
  8820,
  8821,
  8822,
  8823,
  8824,
  8825,
  23140],
 [1235,
  10642,
  11821,
  17277,
  19916,
  20691,
  20700,
  21304,
  21305,
  21306,
  21307,
  21308,
  21309,
  21310,
  21311,
  21312,
  21313,
  21314,
  21315,
  21316,
  21317,
  21318,
  21319,
  21320,
  21321,
  21322,
  21323,
  21324,
  21325,
  21326,
  21327,
  21328,
  21329,
  21330,
  21331,
  21332,
  21333,
  21334,
  21335,
  21336,
  21337,
  21338,
  21339,
  21340,
  21341,
  21342,
  21343,
  21344,
  21345,
  21346,
  213

In [41]:
com_nodes = [group for group in coms.communities if get_charity_by_name(G, 'ACMI Migrant Fund') in group]

In [42]:
def get_community_subgraph_visualise2(graph, charity_name, community_nodes, filename):
    charity_node = get_charity_by_name(graph, charity_name)
    subgraph = graph.subgraph(community_nodes)

    visualise_subgraph(subgraph, charity_node, filename)
    return 

In [43]:
for i in range(len(com_nodes)):
    get_community_subgraph_visualise2(G, 'ACMI Migrant Fund', com_nodes[i], f'acmi_community_dcs{i+1}')

../graph/acmi_community_dcs1.html
../graph/acmi_community_dcs2.html
../graph/acmi_community_dcs3.html


Using the same example of ACMI Migrant Fund, this algorithm categorises the charity into 3 communities, showing the overlaps in communities.

Visually inspecting the graphs, the communities appear much smaller compared to the ones created by the Ego-Splitting. Additionally, for this network, it ran significantly longer at 3 minutes compared to a few seconds for the Ego-Splitting.

### Parallel Louvain Method

This is a parallel implementation of the Louvain method, which optimises for modularity on nodes and communities. Communities are formed on the basis that each community has denser connection within the community than between communities. This method works for non-overlapping clusters.

Source: Blondel, V.D. et al. Fast unfolding of communities in large networks. J. Stat. Mech 10008, 1-12(2008). https://doi.org/10.1088/1742-5468/2008/10/P10008

In [44]:
# Convert the NetworkX graph to NetworKit
G_nk = networkit.nxadapter.nx2nk(G)

In [45]:
coms_plm = networkit.community.detectCommunities(G_nk, algo=networkit.community.PLM(G_nk, True))

Communities detected in 0.01411 [s]
solution properties:
-------------------  -----------
# communities        1026
min community size      1
max community size    680
avg. community size    24.1287
imbalance              27.2
edge cut              862
edge cut (portion)      0.03326
modularity              0.959255
-------------------  -----------


In [46]:
get_community_subgraph_visualise2(G, 'UMAR PULAVAR SCHOLARSHIP TRUST FUND', list(coms_plm.getMembers(coms_plm.subsetOf(get_charity_by_name(G, 'UMAR PULAVAR SCHOLARSHIP TRUST FUND')))), 'test_community_plm')

../graph/test_community_plm.html


In [47]:
get_community_subgraph_visualise2(G, 'ACMI Migrant Fund', list(coms_plm.getMembers(coms_plm.subsetOf(get_charity_by_name(G, 'ACMI Migrant Fund')))), 'acmi_community_plm')

../graph/acmi_community_plm.html


In [48]:
get_community_subgraph_visualise2(G, 'IMPART LTD.', list(coms_plm.getMembers(coms_plm.subsetOf(get_charity_by_name(G, 'IMPART LTD.')))), 'impart_community_plm')

../graph/impart_community_plm.html


Visually inspecting the graphs, the ACMI Migrant Fund had similar community graph with the Ego-Splitting version, whereas for IMPART LTD., the PLM graph is much far reaching.

---

## Step 3: Evaluation Metrics

We explore using two metrics to evaluate the different community detection methods. As the community detection methods are used in an unsupervised manner for this network, metrics that rely on ground-truth are not used. Instead, metrics that may indicate the quality of the created communities are used.

**Density**: Represents the proportion of possible relationships in the network that are actually present.

**Modularity**: Measure of relationship of nodes within and outside a community. High modularity indicates dense connections within the community and sparse connections between communities.

Sources: 
- https://neo4j.com/docs/graph-data-science/current/algorithms/modularity/
- [Evaluating Community Detection Algorithms for
Progressively Evolving Graphs
](https://hal.science/hal-03173685/document)

### Original Graph

The original graph without the community detection is not densely connected and contains disjoint components.

Calculate the average density and modularity of each connected component of the graph as the baseline metric.

In [50]:
comps = list(nx.components.connected_components(G))

In [51]:
len(comps)

941

In [52]:
original_total_density = 0
for c in range(len(comps)):
    original_total_density += nx.density(G.subgraph(list(comps[c])))

In [72]:
print(f'Original average density of connected component: {original_total_density / len(comps)}')

Original average density of connected component: 0.24445392444490033


In [75]:
print(f'Original modularity of connected component: {nx.community.modularity(G, comps)}')

Original modularity of connected component: 0.515995032351851


### Ego-Splitting

In [54]:
dict_partions = {}
for node, community in communities.items():
    if community[0] in dict_partions:
        dict_partions[community[0]].append(node)
    else:
        dict_partions[community[0]] = [node]

In [55]:
list_partitions = [value for key, value in dict_partions.items()]

In [56]:
len(list_partitions)

1064

In [57]:
total_density = 0
for i in range(len(list_partitions)):
    total_density += nx.density(G.subgraph(list_partitions[i]))

In [74]:
print(f'Average density of communities from Ego-Splitting method: {total_density / len(list_partitions)}')

Average density of communities from Ego-Splitting method: 0.21874118390346076


In [76]:
print(f'Modularity of communities from Ego-Splitting method: {nx.community.modularity(G, list_partitions)}')

Modularity of communities from Ego-Splitting method: 0.9578064062441182


### Divide and Conquer

In [60]:
len(coms.communities)

2639

In [61]:
total_density_dcs = 0
for i in range(len(coms.communities)):
    total_density_dcs += nx.density(G.subgraph(coms.communities[i]))

In [79]:
print(f'Average density of communities from Divide and Conquer method: {total_density_dcs / len(coms.communities)}')

Average density of communities from Divide and Conquer method: 0.22013157973606476


In [80]:
print(f'Modularity of communities from Divide and Conquer method: {evaluation.modularity_overlap(G, coms).score}')

Modularity of communities from Divide and Conquer method: 0.18262903900975913


### Parallel Louvain Method

In [64]:
len(coms_plm.getSubsetIds())

1026

In [65]:
community_ids = list(coms_plm.getSubsetIds())

In [66]:
total_density_plm = 0
list_partitions_plm = []
for i in range(len(community_ids)):
    list_members = list(coms_plm.getMembers(community_ids[i]))
    total_density_plm += nx.density(G.subgraph(list_members))
    list_partitions_plm.append(list_members)


In [82]:
print(f'Average density of communities from Parallel Louvain method: {total_density_plm / len(community_ids)}')

Average density of communities from Parallel Louvain method: 0.22543418467089574


In [81]:
print(f'Modularity of communities from Parallel Louvain method: {nx.community.modularity(G, list_partitions_plm)}')

Modularity of communities from Parallel Louvain method: 0.9592545239957949


## Part 4: Summary

This notebook is exploratory and has covered a few community detection methods implemented across different libraries. The methods explored here are divisive methods that removes edges and clusters of more densely connected nodes are formed.

The aim of community detection in this network is to identify potential groupings of people and organisations with similar interests, given a network that is disjoint and often times not dense as connections between organisations might be limited.

Visual inspection is done on some communities determined by each method. Of which, the Ego-Splitting Framework, which also uses the Louvain method as its initial steps, appears favourable for this project's problem statement as it provides a good balance between forming communities with related organisations without stretching the community into too many degrees. Curiously, it did not identify any node which belonged in more than one community, despite the method allowing for overlapping.

Further evaluation using modularity across the network communities showed that the Parallel Louvain and Ego-Splitting network had the highest modularity, which could indicate that the community formed by this method has better quality for this network. Both Ego-Splitting and Parallel Louvain method had much higher modularity than the original graph. This means that the communities produced have better connectivity within the community than across communities. Divide and Conquer had even lower modularity than the baseline, which is further supported by the fact that it had the most communities formed by a large margin and visually appeared to cut the network too aggressively into small partitions. Density may also have been a less valuable metric for this network as the original graph already started with low density, and density across all three methods explored did not differ significantly.